In [1]:

# @title Văn bản tiêu đề mặc định
import requests # web: html + css
from bs4 import BeautifulSoup as bs
import random
import pandas as pd
from time import sleep
from selenium import webdriver # web: html + css + javasricpt
from selenium.webdriver.common.keys import Keys #Keys: cung cấp các phím trên bàn phím
from selenium.webdriver.common.by import By # By: định vị các thành phần trong tài liệu.
from selenium.webdriver.chrome.service import Service
import os
import urllib3 # download
from urllib.parse import urlparse, urljoin
import pyautogui as pag, sys # điều khiển bàn phím và chuột 
import pyperclip as ppc # copy/parse
import re
import wget
from webdriver_manager.chrome import ChromeDriverManager


In [2]:
key = input("Enter key to search: ").strip().replace(" ", "%20")
url = f"https://www.pinterest.com/search/pins/?q={key}&rs=typed"

# thiết lập url và kiểm tra kết nối
# url = "https://www.pinterest.com/search/pins/?q=laptop%20dell&rs=typed" # địa chỉ web cần crawl data
r = requests.get(url) # # Send an HTTP request to the URL and lấy data chưa qua xử lý
print("Status code:", r.status_code)
print("Content:", type(r.content))
print("Content:", r.content)


Status code: 200
Content: <class 'bytes'>
Content: b'<!DOCTYPE html><html class="en" lang="en"><head><meta charSet="utf-8"/><meta name="viewport" id="viewport" content="width=device-width, initial-scale=1"/><meta id="csp" http-equiv="Content-Security-Policy" content="default-src &#x27;self&#x27; blob: s.pinimg.com; font-src &#x27;self&#x27; s.pinimg.com data: fonts.googleapis.com fonts.gstatic.com use.typekit.net; style-src &#x27;self&#x27; blob: &#x27;unsafe-inline&#x27; data: *.pinimg.com *.pinterest.com accounts.google.com fonts.googleapis.com; img-src blob: data: *; base-uri &#x27;none&#x27;; connect-src &#x27;self&#x27; blob: *.pinimg.com *.pinterest.com accounts.google.com *.facebook.com www.googleapis.com *.dropboxapi.com pinterest-media-upload.s3.amazonaws.com pinterest-media-upload.s3.us-east-1.amazonaws.com pinterest-media-upload.s3-accelerate.amazonaws.com pinterest-media-upload.s3-accelerate.us-east-1.amazonaws.com *.cedexis.com *.cedexis-radar.net *.tvpixel.com api.pinadmi

In [3]:

# open it, go to a website, and get results
wd = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
# wd = webdriver.Chrome()
wd.get(url) #điều hướng đến trang được cung cấp bởi URL.
print(len(wd.page_source)) #
print(type(wd.page_source))  # <class 'str'>


307519
<class 'str'>


In [4]:
# tạo thư mục làm việc
working_path=r"D:\anh\download\craw-data\pinterest"
# working_path = r"D:\downloads\craw_data\pinterest"
os.chdir(working_path)
print(os.getcwd())


D:\anh\download\craw-data\pinterest


In [5]:
def stop_next():
    print("Function stop_next running")
    while(True):
        sleep(0.5)
        key = input("Enter 'ok' to next: ").strip().lower()
        if key == "ok":
            print("Program is continuing")
            return


In [6]:
def sleepRandom(start=1, end=3, phay=2): # hàm random thời gian ngủ để qua khâu kiểm duyệt 
    # start/end: random time from start -> end 
    # phay: phần trăm của 1 số vd phay=2 -> 2.22
    phantram = 10**phay
    thoigian = random.randint(start*phantram, end*phantram)/phantram
    sleep(thoigian)


In [7]:

# hàm ghi file: đường dẫn, văn bản
def writeFileTxt(path_file, text, mode='w', encoding='UTF-8'):
  print('function: writeFileTxt running')
  # Check if the text is a set
  if isinstance(text, set):
    text = list(text)
  # Check if the text is a list/tuple.
  if isinstance(text, (list, tuple)):
    text = '\n'.join(text)
  with open(path_file, mode=mode, encoding=encoding) as file:
    file.write(text)
    print("write text:", text)
  print('Finished function: writeFileTxt')



In [8]:

# hàm download file: # url / path to fordel -> mode = 1/file -> mode = 0
def downloadFile(url, path_file, mode=0):
  print('function: downloadFile running')
  response = requests.get(url) # send request to url and receive result
  if response.status_code == 200: # check status connection: 200 is OK
    if mode == 1:
      name_file = os.path.basename(urlparse(url).path)
      path_file = os.path.join(path_file, name_file)
    dir_name = os.path.dirname(path_file)
    if dir_name != "" and not os.path.exists(dir_name):
      os.makedirs(dir_name)
    with open(path_file, 'wb') as file: # write file jpg/png
        file.write(response.content)
    print(f"Downloaded: {url}")
  else:
    print(f"Failed to download: {url}")
  response.close()
  sleepRandom()
  print('Finished function: downloadFile')


# hàm download file Pro: # url / path to fordel -> mode = 1/file -> mode = 0: pdf/mp3/mp4
def downloadFilePro(url, path_file, mode=0):
  print('function: downloadFilePro running')
  try:
    if mode == 1:
      name_file = os.path.basename(urlparse(url).path)
      path_file = os.path.join(path_file, name_file)
    dir_name = os.path.dirname(path_file)
    if dir_name != "" and not os.path.exists(dir_name):
      os.makedirs(dir_name)
    wget.download(url, out=path_file)
    print(f"Downloaded: {url}")
  except Exception as e:
    print(f"Failed to download: {url}")
    print(e)
  sleepRandom()
  print('Finished function: downloadFilePro')

In [9]:
def remove_special_characters(input_string):
    # Sử dụng biểu thức chính quy để chỉ giữ lại các ký tự chữ và số, cùng khoảng trắng
    cleaned_string = re.sub(r'[^a-zA-Z0-9\sàáạãảâầấậẫẩăằắặẵẳèéẹẽẻêềếệễểìíịĩỉòóọõỏôồốộỗổơờớợỡởùúụũủưừứựữửỳýỵỹỷđđ]', '', input_string)
    return cleaned_string

In [10]:
imagelist = set()
def get_image_list():
    sleepRandom()
    # định dạng và lấy data bằng thư viện BeautifulSoup
    html_doc = wd.page_source
    soup = bs(html_doc, 'html.parser')
    div_images = set(soup.find_all('div', {"data-test-id": "pin-visual-wrapper"}))
    return div_images

while(len(imagelist)<=30):
    imagelist = imagelist.union(get_image_list())
    # Cuộn xuống cuối trang
    # wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    wd.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
wd.quit()
print(len(imagelist))
for i in imagelist:
    print(i)

42
<div class="XiG zI7 iyn Hsu" data-test-id="pin-visual-wrapper" style="margin-top: 0%; margin-bottom: 0%;"><div class="Pj7 sLG XiG ho- m1e"><div class="XiG zI7 iyn Hsu" style="background-color: rgb(121, 86, 64); padding-bottom: 90.2542%;"><img alt="Cartoons Laptop Skin Sticker Cute Notebook Vinyl Decal Dell Hp | Etsy" class="hCL kVc L4E MIw" fetchpriority="auto" loading="auto" src="https://i.pinimg.com/236x/32/9e/91/329e9167e2fc771f2a042ee892ad0b63.jpg"/></div><div class="KPc MIw ojN Rym p6V QLY"></div></div></div>
<div class="XiG zI7 iyn Hsu" data-test-id="pin-visual-wrapper" style="margin-top: 0%; margin-bottom: 0%;"><div class="Pj7 sLG XiG ho- m1e"><div class="XiG zI7 iyn Hsu" style="background-color: rgb(153, 102, 96); padding-bottom: 68.6441%;"><img alt="Is the Dell XPS 13 the best Windows laptop around? | Engadget" class="hCL kVc L4E MIw" fetchpriority="auto" loading="auto" src="https://i.pinimg.com/236x/8e/ff/94/8eff942e811bcd4dd6dba0a0d93a4c89.jpg"/></div><div class="KPc MIw 

In [82]:
folder_fath = 'image'
number = 0
for img in imagelist:
    img_tab = img.find('img', {'src': True})
    src = img_tab.get('src')
    name = str(img_tab.get('alt')).strip()
    # Loại bỏ các ký tự không phải chữ cái, số and Thay thế dấu cách bằng dấu "-"
    name = remove_special_characters(name).replace(' ', '-')
    print('src:', src)
    print('name:', name)
    if len(name)>30:
        name = name[0:30]
    path_file = os.path.join(folder_fath, name + str(number) + '.jpg')
    print(path_file)
    downloadFile(src, path_file)
    number += 1
    

src: https://i.pinimg.com/236x/06/06/37/0606374d5f3bde41e051cdc5242bf792.jpg
name: Mô-hình-con-rồng-xốp-trang-trí-Tết-Giáp-Thìn-2024
image\Mô-hình-con-rồng-xốp-trang-trí0.jpg
function: downloadFile running
Downloaded: https://i.pinimg.com/236x/06/06/37/0606374d5f3bde41e051cdc5242bf792.jpg
Finished function: downloadFile
src: https://i.pinimg.com/236x/ef/04/8e/ef048e3276812abe93f4410f93fbe88c.jpg
name: Bộ-Số-2024--Con-Rồng-Trang-Trí-Tết-Giáp-Thìn-007
image\Bộ-Số-2024--Con-Rồng-Trang-Trí1.jpg
function: downloadFile running
Downloaded: https://i.pinimg.com/236x/ef/04/8e/ef048e3276812abe93f4410f93fbe88c.jpg
Finished function: downloadFile
src: https://i.pinimg.com/236x/45/ba/de/45badec7ae41248d4d26c3f6f16bd8c0.jpg
name: Tạo-avatar-Tết-2024-xuân-Giáp-Thìn-cực-đẹp
image\Tạo-avatar-Tết-2024-xuân-Giáp-2.jpg
function: downloadFile running
Downloaded: https://i.pinimg.com/236x/45/ba/de/45badec7ae41248d4d26c3f6f16bd8c0.jpg
Finished function: downloadFile
src: https://i.pinimg.com/236x/7d/ee/e1/7d

In [ ]:
# ------------------------- start program here ---------------------------------

In [ ]:
# Cuộn xuống cuối trang
wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [ ]:
# định dạng và lấy data bằng thư viện BeautifulSoup
html_doc = wd.page_source
soup = bs(html_doc, 'html.parser')
imagelist = soup.find_all('div', {"data-test-id": "pin-visual-wrapper"})
imagelist

In [38]:
folder = os.path.dirname('')
folder

'img'

In [24]:
# Tìm và bấm nút mở hộp thoại
button = driver.find_element_by_id("open-alert-button")
button.click()

# Chờ đến khi hộp thoại xuất hiện (có thể sử dụng WebDriverWait)
alert = driver.switch_to.alert

# In nội dung của hộp thoại
print("Alert Text:", alert.text)

# Chấp nhận hộp thoại (đóng nó)
alert.accept()

100 câu chúc Tết hay và ý nghĩa cho năm Giáp Thìn 2024


In [60]:
s1 = set([1, 2, 5])
s2 = set([1, 2, 3, 4])
result = s1.union(s2)
print(len(result))

5


In [ ]:
# định dạng và lấy data bằng thư viện BeautifulSoup
html_doc = wd.page_source
soup = bs(html_doc, 'html.parser')

In [ ]:
imagelist = soup.find_all('div', {"data-test-id": "pin-visual-wrapper"})
imagelist

In [ ]:
email = 'lea81807@gmail.com'
password = '123456789anhs.'
click <a> kham pha 

In [ ]:
# nhập từ khóa từ input -> url + key -> mở bằn selenium -> lấy ảnh về: tìm <div> chứa ảnh -> <a>/<img>

In [ ]:
sleep(5)
print(pag.position())

In [ ]:
a = [1,2]
b = [3]
b.extend(a)
b

In [ ]:
sleepRandom()
wd.quit()